In [90]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from collections import defaultdict
import json
from langchain_text_splitters import RecursiveJsonSplitter

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [91]:
embedding_function = OpenAIEmbeddings()

# Create the language model
llm = ChatOpenAI(model='gpt-4o', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [92]:
def csv_files_to_json(file_paths, encoding='cp949'):
    # Initialize a default dictionary to store dictionaries for each ID
    json_data = defaultdict(lambda: defaultdict(list))
    
    # Iterate through each CSV file
    for file_path in file_paths:
        # Extract the file name without extension to use as a key
        file_key = file_path.split('/')[-1].split('.')[0]
        
        # Read the CSV file into a DataFrame with cp949 encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Group the data by 'id'
        for id, group in df.groupby('id'):
            # Convert the group data to a dictionary without the 'id' column
            record = group.drop(columns=['id']).to_dict(orient='records')
            # Append the record under the corresponding file name for the given id
            json_data[str(id)][file_key].extend(record)
    
    # Convert the default dictionary to a regular dictionary and then to a JSON string
    json_str = json.dumps(dict(json_data), indent=4)
    
    return json_str

In [93]:
def inbody_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/inbody.csv']
    
    inbody_json = csv_files_to_json(file_paths)
    json_data = json.loads(inbody_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 인바디 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
        
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    # texts = splitter.split_text(json_data=json_data)

    #벡터스토어 생성
    inbody_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    inbody_retriever = inbody_vectorstore.as_retriever()
#     inbody_query = f"""
#     context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이며 context의 pat_sex는 환자의 성별, pat_birth는 환자의 탄생연도를 뜻한다.\n\n
#     """
#     
#     inbody_query += "{context}"
#     prompt = ChatPromptTemplate.from_messages(
# [
#     (
#     "system",
#     inbody_query,
#      ),
#     ("human", "{question}")
#     ]
# )
    
    inbody_chain = (
        {"context": inbody_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    inbody_response = inbody_chain.invoke(question)
    return inbody_response

In [94]:
def vital_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/vital.csv']
    
    vital_json = csv_files_to_json(file_paths)
    json_data = json.loads(vital_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 바이탈 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    vital_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    vital_retriever = vital_vectorstore.as_retriever()
#     vital_query = f"""
#     context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이며 context의 pat_sex는 환자의 성별, pat_birth는 환자의 탄생연도를 뜻한다.\n\n
#     """
#     
#     vital_query += "{context}"
#     prompt = ChatPromptTemplate.from_messages(
# [
#     (
#     "system",
#     vital_query,
#      ),
#     ("human", "{question}")
#     ]
# )
    
    vital_chain = (
        {"context": vital_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    vital_response = vital_chain.invoke(question)
    return vital_response

In [95]:
def patients_analysis(embedding_function, llm, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/patients.csv']
    
    patients_json = csv_files_to_json(file_paths)
    json_data = json.loads(patients_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 환자 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    patients_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    patients_retriever = patients_vectorstore.as_retriever()
    
    patients_query = f"""
    context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이며 context의 pat_sex는 환자의 성별, pat_birth는 환자의 탄생연도를 뜻한다.\n\n
    """
    
    patients_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    patients_query,
     ),
    ("human", "{question}")
    ]
)
    
    patients_chain = (
        {"context": patients_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    patients_response = patients_chain.invoke(question)
    return patients_response

In [96]:
def reports_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/reports.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/reports.csv']
    
    reports_json = csv_files_to_json(file_paths)
    json_data = json.loads(reports_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 설문 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    reports_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    reports_retriever = reports_vectorstore.as_retriever()
    
#     reports_query = f"""
#     context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이며 context의 pat_sex는 환자의 성별, pat_birth는 환자의 탄생연도를 뜻한다.\n\n
#     """
#     
#     reports_query += "{context}"
#     prompt = ChatPromptTemplate.from_messages(
# [
#     (
#     "system",
#     reports_query,
#      ),
#     ("human", "{question}")
#     ]
# )
    
    reports_chain = (
        {"context": reports_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    reports_response = reports_chain.invoke(question)
    return reports_response

In [97]:
def final_response(id, question):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id, question)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id, question)
    patients_response = patients_analysis(embedding_function, llm, id, question)
    reports_resonse = reports_analysis(embedding_function, llm, analysis_prompt, id, question)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    """
    
    final_query = f"""
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{total_analysis}"""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(question)
    return response

In [99]:
print(final_response('7','건강정보에 대한 종합적인 소견을 말해'))

7에 대한 인바디 정보 분석 중...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
print(final_response('1','추천하는 운동을 알려줘'))

In [ ]:
print(final_response('1','추천하는 식단들을 설명해'))